参考: https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake-gen2.html#mount-an-adls-filesystem-to-dbfs-using-a-service-principal-and-oauth-2-0


### 手順
1. Service Principal を作成する
2. Service Principal に Aure リソースの権限をあたえる
3. Key Vault に Service Principal のシークレットを登録する
4. Azure Dtabricks の Secret Scope を登録する 
5. Service Principal の Object ID を取得する 
6. ADLS Gen2 のディレクトリに 5. で取得した ID を使って権限を設定する

5 のコマンド
```az ad sp show --id <application id> --query objectId```

In [2]:
#Service Principal の設定
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "<application id>",
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "<secret scope name>", key = "<key vault secret name for service principal secret>"),
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant id>/oauth2/token"}

In [3]:
# dbutils.fs.unmount(mount_point = "/mnt/adlsgen2")

In [4]:
dbutils.fs.mount(
  source = "abfss://<file system>@<storage account name>.dfs.core.windows.net/[<path>]",
  mount_point = "<dbfs mount point>",
  extra_configs = configs)

In [5]:
df = spark.read.csv("<dbfs file path>/<filename>", header = True, inferSchema = True)

In [6]:
#SQL DW の JDBC 接続を定義
jdbc_connection_string = dbutils.secrets.get("<secret scope name>", "<key vault secret name for sql dw jdbc connection string>")

In [7]:
1#一時領域で使用するBlobの設定
spark.conf.set("fs.azure.account.key.<Storage Account Name>.blob.core.windows.net",  dbutils.secrets.get("<secret scope name>", "<key vault for storage account name>"))

In [8]:
#SQL DW に書き込む
#Table は事前に作成する必要なし
df_out.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", jdbc_connection_string) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbtable", "<table name>") \
  .option("tempdir", "wasbs://adbtmp@kaokurblob.blob.core.windows.net/tmp") \
  .save()